# Analyze results
Aug 28, 2020

In [12]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize

import pandas as pd

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis/')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [5]:
# Read input data

fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
samples=np.load(fname,mmap_mode='r+')[:2000]
samples=samples[:,0,:,:]
print(samples.shape)

main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
results_dir=main_dir+'20200910_063316'

### Read loss data
with open (results_dir+'/metrics.pickle','rb') as f:
    metrics_dict=pickle.load(f)


(2000, 128, 128)


## Plot Losses

In [7]:
df_metrics=pd.DataFrame([])
for key in metrics_dict.keys(): df_metrics[key]=metrics_dict[key]
df_metrics['step']=np.arange(df_metrics.shape[0])

In [48]:
df_metrics.step.values.shape,df_metrics.G.values.shape

((16000,), (16000,))

In [8]:
df_metrics.head(10)

,Dreal,Dfake,Dfull,G,spec_chi,hist_chi,step
0,0.790947,0.645672,1.436619,2.808769,0.287756,-6.096606,0
1,0.265613,0.928163,1.193776,3.348862,0.287756,-6.095263,1
2,0.608416,0.339964,0.948380,2.975842,0.287756,-6.089023,2
3,0.252395,0.566424,0.818820,4.372735,0.287756,-6.113003,3
4,0.311941,0.251073,0.563013,4.457201,0.287756,-6.152032,4
5,0.133358,0.462941,0.596298,6.025072,0.287756,-6.158807,5
6,0.231155,0.199754,0.430910,5.805678,0.287756,-6.164237,6
7,0.130584,0.392675,0.523259,7.831878,0.287756,-6.196768,7
8,0.189747,0.036211,0.225959,5.867364,0.287756,-6.185212,8
9,0.041869,0.768805,0.810674,11.215940,0.287756,-6.250702,9


In [14]:
plt.figure()
for key in ['G','Dfull','Dfake','Dreal']:
    plt.plot(df_metrics[key],label=key)

plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
df_metrics.plot(kind='line',x='step',y=['spec_chi','hist_chi'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
chi=df_metrics.quantile(q=0.01,axis=0)['spec_chi']
print(chi)
df_metrics[df_metrics.spec_chi<chi].sort_values(by=['spec_chi'])

0.28775550966786767


,Dreal,Dfake,Dfull,G,spec_chi,hist_chi,step


In [18]:
### Read GAN results
# epoch,step=0,100
# fname='gen_img_epoch-{0}_step-{1}.npy'.format(epoch,step)

step=5000
fname='gen_img_epoch-*_step-{0}.npy'.format(step)
fle=glob.glob(results_dir+'/images/'+fname)[0]
fle
imgs=np.load(fle)
print(imgs.shape)

(128, 128, 128)


### Plot images

In [19]:
f_plot_grid(samples[100:118],cols=6,fig_size=(6,3))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
f_plot_grid(imgs[10:28],cols=6,fig_size=(6,3))

3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot Histogram

In [24]:
img_lst=[imgs]
f_compare_pixel_intensity(img_lst,label_lst=['imgs'],bkgnd_arr=samples,log_scale=True, normalize=True, mode='avg',bins=25, hist_range=None)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot spectrum

In [22]:
img_lst=[imgs]
f_compare_spectrum(img_lst,label_lst=['img'],bkgnd_arr=samples,log_scale=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Read stored chi-squares for images

In [13]:
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20200914_084532'
df=pd.read_pickle(main_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)

In [14]:
df

,epoch,step,img_type,fname,chi_1a,chi_1b,chi_1c,chi_1,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_large,num_vlarge,hist_val,hist_err,hist_bin_centers,spec_val,spec_err
0,0,0,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,135.476644,0.214317,0.013186,135.704147,11.233163,15.171317,53021.110764,1.468973e+08,0,0,"[0.0004562437546100468, 0.10093273237994162, 0...","[4.3211298623102846e-05, 0.002832351229120047,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[66413480.259679615, 690.6281577019254, 495.84...","[16063.885818430459, 22.150576520183918, 13.76..."
1,0,50,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,3.580642,0.182874,0.013186,3.776701,1.944219,3.876160,12411.706291,1.783004e+08,0,0,"[0.20630385034845525, 1.7724988040809575, 0.86...","[0.0021938429206743585, 0.004108243170571216, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[153579394.6977463, 15761.517857417464, 4742.9...","[17191.021196918948, 60.486987183492175, 19.39..."
2,0,100,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,4.681509,0.294280,0.013186,4.988976,1.598552,3.914359,2500.317340,2.471524e+07,0,0,"[0.30513225574520014, 1.477058979163924, 0.909...","[0.005598496166707251, 0.004181018671766911, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[153365213.92568398, 103454.97341382504, 43442...","[226537.8423326675, 504.74157536151466, 125.32..."
3,0,150,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,2.680132,0.294280,0.013186,2.987598,1.222999,15.510213,3026.568441,3.512653e+07,0,0,"[0.4453145575088736, 1.5858103508343218, 0.826...","[0.02285723062834215, 0.012827321959654626, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[170160294.54405886, 99975.09070710465, 39647....","[1676815.9572969612, 3248.8780960982954, 1084...."
4,0,200,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.042947,0.160394,0.013186,0.216526,0.010456,7.272453,2798.682104,3.595637e+07,0,0,"[1.1267239927505543, 0.8932293816259115, 0.292...","[0.007879323030371659, 0.00799375479159793, 0....","[-1.031746031584782, -0.6161616169043975, -0.3...","[198315132.69267416, 196436.46939581633, 19999...","[515128.21764971537, 7108.456653699832, 7291.4..."
5,0,250,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.190200,0.059893,0.011844,0.261937,0.186184,5.305885,175.152482,2.220154e+06,0,0,"[1.362480912146614, 0.6070126006416946, 0.2242...","[0.0032764703586943564, 0.0036224009147602446,...","[-1.031746031584782, -0.6161616169043975, -0.3...","[212305721.0068456, 103554.98085502535, 108939...","[271850.54603376874, 3146.9090922807704, 2391...."
6,0,300,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.581002,0.194086,0.003179,0.778267,0.329319,12.014207,88.337564,1.028065e+06,0,0,"[1.3993925339316189, 0.47572879566518, 0.20923...","[0.010235986180788414, 0.0067426426546524565, ...","[-1.031746031584782, -0.6161616169043975, -0.3...","[210012283.47778594, 110293.4830712229, 107462...","[971352.2781738021, 3480.2147381506097, 3414.6..."
7,0,350,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.458388,0.013878,0.016468,0.488734,0.422466,8.641284,126.400511,1.458562e+06,0,0,"[1.5071083048116831, 0.449472919798853, 0.1665...","[0.008546749868410959, 0.007450865581527171, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[222491102.47252572, 65616.81744980812, 72027....","[722640.6711692692, 2812.251941618422, 2331.33..."
8,0,400,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.301021,0.002366,0.001626,0.305013,0.267539,12.631489,14.152059,8.063583e+04,0,0,"[1.3893523550107245, 0.5226155527535863, 0.226...","[0.015335712360779085, 0.012576078661472456, 0...","[-1.031746031584782, -0.6161616169043975, -0.3...","[213892283.0285019, 112639.02898454666, 67633....","[1245426.2246434756, 2546.508572892161, 1566.9..."
9,0,450,train,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,0.768900,0.256110,0.013186,1.038196,0.498360,5.006125,135.202617,1.566218e+06,0,0,"[1.5694165568301561, 0.